In [1]:
import tensorflow as tf

print("TensorFlow Version:", tf.__version__)
print("Devices Available:", tf.config.list_physical_devices())
print("GPU Available:", tf.config.list_physical_devices('GPU'))


TensorFlow Version: 2.16.2
Devices Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import TextVectorization

# ---------------------------------------------------------------
# 1. Load and Preprocess the IMDb Dataset
# ---------------------------------------------------------------
# Load the IMDb dataset and split into training, validation, and test sets
train_data, val_data, test_data = tfds.load(
    'imdb_reviews',
    split=['train[:80%]', 'train[80%:]', 'test'],
    as_supervised=True,
    with_info=False
)

# Constants
MAX_FEATURES = 10000  # Vocabulary size
MAX_LEN = 200  # Maximum sequence length
BATCH_SIZE = 256  # Batch size for better GPU utilization

# TextVectorization Layer (on CPU)
with tf.device('/CPU:0'):
    vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=MAX_LEN)
    text_ds = train_data.map(lambda x, y: x).take(1000).cache()  # Use a subset to adapt the vocabulary
    vectorizer.adapt(text_ds)

# Preprocessing function for datasets
def preprocess_text(text, label):
    return vectorizer(text), tf.cast(label, tf.int32)

# Preprocess datasets
train_dataset = (
    train_data
    .map(preprocess_text, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

val_dataset = (
    val_data
    .map(preprocess_text, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

test_dataset = (
    test_data
    .map(preprocess_text, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# ---------------------------------------------------------------
# 2. Build the Conv1D-Based Model
# ---------------------------------------------------------------
# Enable mixed precision for faster training on Apple GPUs
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Define the Conv1D model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(MAX_LEN,)),  # Explicit input shape
    tf.keras.layers.Embedding(MAX_FEATURES, 128),  # Embedding layer
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.GlobalMaxPooling1D(),  # Global pooling to reduce dimensions
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout for regularization
    tf.keras.layers.Dense(1, activation='sigmoid', dtype='float32')  # Output layer in float32 for stability
])

# Compile the model with mixed precision optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ---------------------------------------------------------------
# 3. Train the Model with Validation Set
# ---------------------------------------------------------------
import time

start_time = time.time()

history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    verbose=2
)

end_time = time.time()
training_time = end_time - start_time

print(f"Training completed in {training_time / 60:.2f} minutes")

# ---------------------------------------------------------------
# 4. Evaluate the Model on Test Data
# ---------------------------------------------------------------
results = model.evaluate(test_dataset, verbose=2)
print(f"Test Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1]:.4f}")

# ---------------------------------------------------------------
# 5. Make Predictions on Test Data
# ---------------------------------------------------------------
for text_batch, label_batch in test_dataset.take(1):  # Take one batch of test data
    predictions = model.predict(text_batch)  # Predict sentiment scores (probabilities)
    
    predicted_labels = tf.where(predictions > 0.5, 1, 0)  # Convert probabilities to binary labels (0 or 1)

    # Print a few examples with their predictions
    for i in range(5):  # Display first 5 samples in the batch
        print(f"Review: {text_batch[i].numpy()[:100]}...")  # Show part of the review (vectorized form)
        print(f"Actual Sentiment: {'Positive' if label_batch[i].numpy() == 1 else 'Negative'}")
        print(f"Predicted Sentiment: {'Positive' if predicted_labels[i].numpy() == 1 else 'Negative'}")
        print("---")


Epoch 1/10


2025-03-31 17:42:40.671331: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-03-31 17:42:40.672480: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


79/79 - 5s - 62ms/step - accuracy: 0.6102 - loss: 0.6386 - val_accuracy: 0.7932 - val_loss: 0.4460
Epoch 2/10
79/79 - 4s - 48ms/step - accuracy: 0.8297 - loss: 0.3824 - val_accuracy: 0.8476 - val_loss: 0.3430
Epoch 3/10
79/79 - 3s - 43ms/step - accuracy: 0.9071 - loss: 0.2382 - val_accuracy: 0.8554 - val_loss: 0.3616
Epoch 4/10
79/79 - 3s - 39ms/step - accuracy: 0.9477 - loss: 0.1508 - val_accuracy: 0.8406 - val_loss: 0.4459
Epoch 5/10
79/79 - 3s - 40ms/step - accuracy: 0.9500 - loss: 0.1310 - val_accuracy: 0.8560 - val_loss: 0.4116
Epoch 6/10
79/79 - 3s - 40ms/step - accuracy: 0.9801 - loss: 0.0649 - val_accuracy: 0.8374 - val_loss: 0.5219
Epoch 7/10
79/79 - 4s - 50ms/step - accuracy: 0.9958 - loss: 0.0222 - val_accuracy: 0.8450 - val_loss: 0.5627
Epoch 8/10
79/79 - 4s - 55ms/step - accuracy: 0.9956 - loss: 0.0179 - val_accuracy: 0.8446 - val_loss: 0.6431
Epoch 9/10
79/79 - 4s - 57ms/step - accuracy: 0.9972 - loss: 0.0122 - val_accuracy: 0.8454 - val_loss: 0.6991
Epoch 10/10
79/79 - 5

2025-03-31 17:43:21.711017: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
